[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/GEE-Courses/blob/master/docs/geemap_intro/11_analysis.ipynb)

## Descriptive statistics

Uncomment and execute the following code block to install geemap if needed.

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(centroid).first()

vis = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(centroid, 8)
Map.addLayer(image, vis, "Landsat-8")
Map

Using Earth Engine built-in functions to get image properties.

In [ ]:
image.propertyNames().getInfo()

In [ ]:
image.get('CLOUD_COVER').getInfo()

Using `geemap.image_props()` to get image properties.

In [ ]:
props = geemap.image_props(image)
props.getInfo()

Get image summary statistics, such as max, min, mean, std, sum.

In [ ]:
stats = geemap.image_stats(image, scale=90)
stats.getInfo()

## Zonal statistics

Add Earth Engine datasets.

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine DEM to map
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add Landsat data to map
landsat = ee.Image('LE7_TOA_5YEAR/1999_2003')

landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat, landsat_vis, "LE7_TOA_5YEAR/1999_2003")

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')
Map

Computer zonal statistics. 
- Allowed output formats: csv, shp, json, kml, kmz
- Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

In [ ]:
out_dem_stats = 'dem_stats.csv'
geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000)

In [ ]:
out_landsat_stats = 'landsat_stats.csv'
geemap.zonal_statistics(
    landsat, states, out_landsat_stats, statistics_type='SUM', scale=1000
)

## Zonal statistics by group

Add Earth Engine datasets.

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, 'US States')
Map.add_legend(builtin_legend='NLCD')
Map

Computer zonal statistics by group.
- statistics_type can be either 'SUM' or 'PERCENTAGE'
- denominator can be used to convert square meters to other areal units, such as square kilimeters.

In [ ]:
nlcd_stats = 'nlcd_stats.csv'

geemap.zonal_statistics_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)